In [2]:
import numpy as np
import pandas as pd 
import time
import math
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.metrics import roc_auc_score, classification_report
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.preprocessing import Imputer, StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from IPython.display import Image
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.mixture import GMM
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
import patsy
from sklearn import neighbors, metrics, svm
from sklearn.cross_validation import StratifiedKFold
from operator import itemgetter
%matplotlib inline 
import seaborn as sns
import xgboost as xgb

In [11]:
import xgboost as xg

ImportError: No module named xgboost

In [47]:
def auc_plotting_function(rate1, rate2, rate1_name, rate2_name, curve_name):
    AUC = auc(rate1, rate2)
    # Plot of a ROC curve 
    plt.figure(figsize=[11,9])
    plt.plot(rate1, rate2, label=curve_name + ' (area = %0.2f)' % AUC, linewidth=4)
    plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel(rate1_name, fontsize = 18)
    plt.ylabel(rate2_name, fontsize = 18)
    plt.title(curve_name + ' Good Customer', fontsize=18)
    plt.legend(loc = "lower right")
    plt.show()
def plot_roc(y_test, y_score):
    fpr, tpr, _ = roc_curve(y_test, y_score)
    auc_plotting_function(fpr, tpr, 'False Positive Rate', 'True Positive Rate', 'ROC')

In [6]:
act = pd.read_csv('/Users/smoot/Downloads/act_train.csv', parse_dates=['date'])
ppl = pd.read_csv('/Users/smoot/Downloads/people.csv', parse_dates=['date'])
# act_t = pd.read_csv("../../../Downloads/act_test.csv", parse_dates=['date'])

In [34]:
# for i in range(1, 11):
#     act['char_' + str(i)].fillna(value = 'type 0', inplace = True)
# for i in range(1, 11):
#     act_t['char_' + str(i)].fillna(value = 'type 0', inplace = True)

In [8]:
act['char_1'].fillna(value = 'type 2', inplace = True)
act['char_2'].fillna(value = 'type 2', inplace = True)
act['char_3'].fillna(value = 'type 1', inplace = True)
act['char_4'].fillna(value = 'type 3', inplace = True)
act['char_5'].fillna(value = 'type 6', inplace = True)
act['char_6'].fillna(value = 'type 2', inplace = True)
act['char_7'].fillna(value = 'type 1', inplace = True)
act['char_8'].fillna(value = 'type 4', inplace = True)
act['char_9'].fillna(value = 'type 8', inplace = True)
act['char_10'].fillna(value = 'type 1', inplace = True)
# act_t['char_1'].fillna(value = 'type 2', inplace = True)
# act_t['char_2'].fillna(value = 'type 2', inplace = True)
# act_t['char_3'].fillna(value = 'type 1', inplace = True)
# act_t['char_4'].fillna(value = 'type 3', inplace = True)
# act_t['char_5'].fillna(value = 'type 6', inplace = True)
# act_t['char_6'].fillna(value = 'type 2', inplace = True)
# act_t['char_7'].fillna(value = 'type 1', inplace = True)
# act_t['char_8'].fillna(value = 'type 4', inplace = True)
# act_t['char_9'].fillna(value = 'type 8', inplace = True)
# act_t['char_10'].fillna(value = 'type 1', inplace = True)

In [9]:
for i in range(1, 10):
    ppl['char_' + str(i)] = ppl['char_' + str(i)].str.lstrip('type ').astype(np.int32) 
for i in range(10, 38):
    ppl['char_' + str(i)] = ppl['char_' + str(i)].astype(np.int32)
    
for i in range(10, 11):
    act['char_' + str(i)] = act['char_' + str(i)].str.lstrip('type ').astype(np.int32)
# for i in range(10, 11):
#     act_t['char_' + str(i)] = act_t['char_' + str(i)].str.lstrip('type ').astype(np.int32)

for table in [act, ppl]:
    table['year'] = table['date'].dt.year
    table['month'] = table['date'].dt.month
    table['day'] = table['date'].dt.day
    
del ppl['date']
del act['date']
y = act['outcome']
del act['outcome']

In [11]:
# act.drop(act.ix[:, 4:13], inplace = True, axis = 1)
# act_t.drop(act_t.ix[:, 4:13], inplace = True, axis = 1)
# act.head()
# act_t.shape

In [13]:
df = pd.merge(act, ppl, how='left', on='people_id')
# df_t = pd.merge(act_t, ppl, how='left', on='people_id')

In [ ]:
# del act['year']
# del act_t['year']


In [14]:
df['people_id'] = df['people_id'].str.lstrip('ppl_ ')
df['peoples_id'] = pd.to_numeric(df['people_id']) 
df['activity_category'] = df['activity_category'].str.lstrip('type ').astype(int, inplace = True)
df['group_1'] = df['group_1'].str.lstrip('group ').astype(np.int32)
del df['people_id']
# del df['group_1']

# df_t['people_id'] = df_t['people_id'].str.lstrip('ppl_ ')
# df_t['peoples_id'] = pd.to_numeric(df_t['people_id']) 
# df_t['activity_category'] = df_t['activity_category'].str.lstrip('type ').astype(int, inplace = True)
# df_t['group_1'] = df_t['group_1'].str.lstrip('group ').astype(np.int32)
# del df_t['people_id']
# del df_t['group_1']

In [ ]:
# del df['group_1']
# del df_t['group_1']
# del df['peoples_id']
# del df_t['peoples_id']


In [15]:
X = pd.DataFrame(df.ix[:, 1:])
# X_t = pd.DataFrame(df_t.ix[:, 1:])
X = pd.get_dummies(X)
# X_t = pd.get_dummies(X_t)

In [55]:
X.drop(X.ix[:, 133:138], inplace = True, axis = 1)

In [17]:
# X.drop(X.ix[:, 156:162], inplace = True, axis = 1)
# for i in np.ravel(X.columns), np.ravel(X_t.columns):
#     if i == np.ravel(X.columns) & np.ravel(X_t.columns):
#         print i
# X_t = X_t.reindex_like(X)
# print np.ravel(X_t.columns)
print df.ix[:, 1:].shape
# print df_t.ix[:, 1:].shape
# print X_t.shape
print X.shape 


(2197291, 57)
(2197291, 206)


In [ ]:
# for table in [act, act_t]:
#     table['year'] = table['date'].dt.year
#     table['month'] = table['date'].dt.month
#     table['day'] = table['date'].dt.day

In [ ]:
# X['f_1'] = X.ix[:,3:4]
# X['f_2'] = X.ix[:,4:5]
# X['f_3'] = X.ix[:,8:9]
# X['f_4'] = X.ix[:,9:10]
# X['f_5'] = X.ix[:,40:41]
# X['f_6'] = X.ix[:,41:42]

# X_t['f_1'] = X_t.ix[:,3:4]
# X_t['f_2'] = X_t.ix[:,4:5]
# X_t['f_3'] = X_t.ix[:,8:9]
# X_t['f_4'] = X_t.ix[:,9:10]
# X_t['f_5'] = X_t.ix[:,40:41]
# X_t['f_6'] = X_t.ix[:,41:42]

# X = X.ix[:, 200:]
# X_t = X_t.ix[:, 200:]

In [ ]:
# X['char38'] = X['char_38']
# del X['char_38']
# X_t['char38'] = X_t['char_38']
# del X_t['char_38']

# sc = MinMaxScaler()
# X['char_38'] = sc.fit_transform(X['char38'])
# X_t['char_38'] = sc.fit_transform(X_t['char38'])
# X_t.drop(X_t.ix[:, 1:10], inplace = True, axis = 1)
# X_t = mnsc.transform(X_t)

In [ ]:
cols = list(df.columns)
cols_t = list(df_t.columns)
del cols[0]
del cols_t[0]

In [18]:
le = LabelEncoder()
y = le.fit_transform(y)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 3)

In [45]:
rf = RandomForestClassifier(max_features = 'sqrt', n_estimators = 15, criterion = 'entropy', n_jobs = -1)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.99067489092641248

In [50]:
proba = etc.predict_proba(X_t)
pp = proba[:,1]

In [51]:
ids = act_t['activity_id']

submission = pd.DataFrame({'activity_id' : ids, 'outcome': pp})
submission.to_csv('submission9.csv', index = None)

In [ ]:
rnf = RandomForestClassifier(max_features = 'sqrt', criterion = 'entropy',  n_estimators = 50, n_jobs = -1)
rnf.fit(X_train, y_train)
rnf.score(X_test, y_test)

In [49]:
etc = ExtraTreesClassifier(max_features = 'sqrt', n_estimators = 15, n_jobs = -1)
etc.fit(X_train, y_train)
etc.score(X_test, y_test)

0.99109361489100256

In [ ]:
dbs = DBSCAN(  )

In [ ]:
importances = etc.feature_importances_
std = np.std([etc.feature_importances_ for tree in etc.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color = "r", yerr = std[indices], align = "center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

In [ ]:
params = [
    {'n_estimators': [15, 50],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True],
    'min_samples_split': [2, 3]}
]

In [ ]:
# from sklearn.grid_search import RandomizedSearchCV
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
# gs = GridSearchCV(rif, param_grid = params, n_jobs = -1)
# gs.fit(X_train, y_train)

In [ ]:
proba = rf.predict_proba(X_t)
ypred = rf.predict(X_t)
pp = proba[:,1]
# score = roc_auc_score(y_test, pp)
# print("Area under ROC {0}".format(score))
# plot_roc(y_test, pp)
# plt.show()

In [ ]:
rf = RandomForestClassifier(max_depth=3, n_estimators = 50, n_jobs = -1)
rf_enc = OneHotEncoder()
rf.fit(X_train, y_train)
rf_enc.fit(rf.apply(X_train))

In [ ]:
# y_pred_rf_lm = rf.predict_proba(rf_enc.transform(rf.apply(X_test)))[:, 1]
y_pred = rf.predict_proba(X_test)
score = rf.score(X_test, y_test)
print score

In [ ]:
# proba = clf.predict_proba(X_test)
# ypred = clf.predict(X_test)
pp = y_pred[:,1]
score = roc_auc_score(y_test, pp)
print("Area under ROC {0}".format(score))

In [ ]:
######################################################################################################################
######################################################################################################################

In [ ]:
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size = 0.16, random_state = 3)

In [ ]:
# logreg_parameters = {
#     'penalty' : ['l2'],
#     'C' : np.array([1, 0.1, 0.01, 0.001]),
#     'solver' : ['liblinear']}
# l_grid = GridSearchCV(lr, param_grid = logreg_parameters, n_jobs = -1)
# l_grid.fit(X_train, y_train)
# lreg_pred = log_grid.predict(X_test)

In [ ]:
lr = LogisticRegression(n_jobs = -1)
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

In [ ]:
y_pp = lr.predict_proba(X_test)
y_pred = lr.predict(X_test)
model_coefs = zip(cols, np.transpose(lr.coef_))

lreg_pp_score = lr.decision_function(X_test)
plot_roc(y_test, lreg_pp_score)
plt.show()

In [ ]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)

In [ ]:
gbc.score(X_test, y_test)

In [ ]:
cv = StratifiedKFold(y, n_folds = 3)
classifier = svm.SVC(kernel = 'linear',
                     random_state = 3)

In [ ]:
probas_ = classifier.fit(X_train, y_train).predict_proba(X_test)

In [ ]:
clf = RandomForestClassifier(n_jobs = -1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
proba = clf.predict_proba(X_test)
ypred = clf.predict(X_test)
pp = proba[:,1]
score = roc_auc_score(y_test, pp)
print("Area under ROC {0}".format(score))

In [ ]:
plot_roc(y_test, pp)
plt.show()

In [ ]:
etc = ExtraTreesClassifier(n_estimators = 100, n_jobs = -1)
etc.fit(X_train, y_train)
etc.score(X_test, y_test)

In [ ]:
prob = etc.predict_proba(X_test)
ya_pred = etc.predict(X_test)
preds_e = prob[:,1]
score = roc_auc_score(y_test, preds_e)
print("Area under ROC {0}".format(score))

In [ ]:
plot_roc(y_test, preds_e)
plt.show()

In [ ]:
ada = AdaBoostClassifier(DecisionTreeClassifier(max_features = 'auto'),
                         algorithm="SAMME.R",
                         n_estimators=50)
ada.fit(X_train, y_train)

In [ ]:
ada.score(X_test, y_test)

In [ ]:
pro = ada.predict_proba(X_test)
yapred = ada.predict(X_test)
preds_a = pro[:,1]
score = roc_auc_score(y_test, preds_a)
print("Area under ROC {0}".format(score))

In [ ]:
pro = etc.predict_proba(X_t)
preds_a = pro[:,1]

# Submission cell

In [ ]:
ids = act_t['activity_id']

submission = pd.DataFrame({'activity_id' : ids, 'outcome': pp})
submission.to_csv('submission7.csv', index = None)

In [ ]:
float(len(df_t.activity_id))

# XGBOOST

In [ ]:
# sum_wpos = sum( weight[i] for i in range(len(label)) if label[i] == 1.0  )

# weight = df.ix[:, 1:] * float(len(df_t)) / len(y)
# stc = StandardScaler()
# X = stc.fit_transform(X)

In [ ]:
# test_count = 550000
# weight = df.ix[:,1:] * float(test_count) / len(label)

In [ ]:
# sum_wpos = []
# if label[i] == 1:
#     sum( weight[i] for i in range(test_count)).append(sum_wpos)
# sum_wneg = []
# if label[i] != 1:
#     sum( weight[i] for i in range(test_count)).append(sum_wneg) 

In [ ]:
# sum_wpos = sum(weight[i] for i in range(len(y)) if y[i] == 1.0)
# sum_wneg = sum(weight[i] for i in range(len(y)) if y[i] == 0.0)
# print ('weight statistics: wpos=%g, wneg=%g, ratio=%g' % ( sum_wpos, sum_wneg, sum_wneg/sum_wpos ))

In [59]:
def evalerror(preds, xgmat):
    labels = xgmat.get_label()
    assert len(preds) == len(labels)
    labels = labels.tolist()
    preds = preds.tolist()
    terms_to_sum = [(math.log(labels[i] + 1) - math.log(max(0,preds[i]) + 1)) ** 2.0 for i,pred in enumerate(labels)]
    return 'error', (sum(terms_to_sum) * (1.0/len(preds))) ** 0.5

In [ ]:
len(df_t.activity_id)

In [ ]:
# weight = df.ix[:,1:] * float(len(df_t.activity_id)) / len(y)

# sum_wpos = sum( weight[i] for i in range(len(y)) if y[i] == 1.0  )
# sum_wneg = sum( weight[i] for i in range(len(y)) if y[i] == 0.0  )

In [ ]:
ids = df_t['activity_id']
test = df_t.drop(['activity_id'],axis = 1)
test_preds = np.zeros(df_t.shape[0])
w = np.random.rand(5, 1)
params = {}
params['objective'] = "binary:logitraw"
params['eta'] = 0.025
params['max_depth'] = 6
params['subsample'] = 0.8
params['colsample_bytree'] = 0.8
params['eval_metric'] = 'auc'
params['alpha'] = 0.2
params['gamma'] = 0.1
params['booster'] = "gbtree"
start_time = time.time()
# params['scale_pos_weight'] = sum_wneg/sum_wpos

param = {}
param['objective'] = 'binary:logitraw'
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'
param['subsample'] = 0.6
param['colsample_bytree'] = 0.6


# 'ams@0.15'
param['silent'] = 1
# param['lambda'] = 0.9
# scale weight of positive examples
# param['scale_pos_weight'] = sum_wneg/sum_wpos


plst = list(params.items())+[('eval_metric', 'ams@0.15')]

X_test, X_train, y_test, y_train = train_test_split(X, y, test_size = 0.2, random_state = 3)
xgmat = xgb.DMatrix(X_train, weight = w, label = y_train)
xgmat_test = xgb.DMatrix(X_test)

watchlist = [ (xgmat,'train') ]
num_rounds = 120

In [ ]:
print ('loading data end, start to boost trees')
bst = xgb.train( param, xgmat, num_rounds, watchlist, feval = evalerror, early_stopping_rounds= 20, verbose_eval = 10 )

In [ ]:
preds = bst.predict(xgmat_test, ntree_limit = bst.best_iteration)
fold_preds = np.around(bst.predict(xgmat_test, ntree_limit = bst.best_iteration), decimals = 1)

In [60]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2197291 entries, 0 to 2197290
Columns: 201 entries, activity_category to char_9_x_type 9
dtypes: float64(154), int32(39), int64(8)
memory usage: 3.0 GB


In [ ]:
data = X
label = y
dtrain = xgb.DMatrix(data, label = label)

In [ ]:
from ml_metrics import rmsle

params = {
params['objective'] = "reg:linear"
params['eta'] = 0.025
params['max_depth'] = 5
params['subsample'] = 0.8
params['colsample_bytree'] = 0.6
params['silent'] = True
}

print ('')

test_preds = np.zeros(test.shape[0])
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test)

watchlist = [(xg_train, 'train')]
num_rounds = 100

xgclassifier = xgb.train(params, xg_train, num_rounds, watchlist, early_stopping_rounds= 20, verbose_eval = 10)
preds = xgclassifier.predict(xg_test, ntree_limit=xgclassifier.best_iteration)

print ('RMSLE Score:', rmsle(y_test, preds))

fxg_test = xgb.DMatrix(test)
fold_preds = np.around(xgclassifier.predict(fxg_test, ntree_limit=xgclassifier.best_iteration), decimals = 1)
test_preds += fold_preds

In [ ]:
#  There are four people who makeup nearly half of the activites.  I'm not sure how to deal with that but this cell 
#  identifies the people and their index in the ppl dataframe.

# r_list = ['ppl_294918', 'ppl_370270', 'ppl_105739', 'ppl_54699']
# for i, l in enumerate(ppl['people_id'] == 'ppl_54699'):
#     if l == True:
#         print i
# for i in r_list:
#     ppl.drop(ppl[ppl['people_id']==i],axis=0)
# [ppl.drop([i], axis = 0, inplace = True) for i in enumerate(ppl.people_id) if i == r_list]
# ppl.drop([102743], inplace = True)
# ppl.drop([142792], inplace = True)
# ppl.drop([3038], inplace = True)
# ppl.drop([165422], inplace = True)

In [ ]:
import numpy as np

import xgboost as xgb

test_size = 550000

# path to where the data lies
dpath = 'data'

# load in training data, directly use numpy
dtrain = np.loadtxt( dpath+'/training.csv', delimiter=',', skiprows=1, converters={32: lambda x:int(x=='s'.encode('utf-8')) } )
print ('finish loading from csv ')

label  = dtrain[:,32]
data   = dtrain[:,1:31]
# rescale weight to make it same as test set
weight = dtrain[:,31] * float(test_size) / len(label)

sum_wpos = sum( weight[i] for i in range(len(label)) if label[i] == 1.0  )
sum_wneg = sum( weight[i] for i in range(len(label)) if label[i] == 0.0  )

# print weight statistics
print ('weight statistics: wpos=%g, wneg=%g, ratio=%g' % ( sum_wpos, sum_wneg, sum_wneg/sum_wpos ))